In [ ]:

from pathlib import Path

! git clone https://github.com/jattardo1/pulse
%cd pulse
import time
from google.colab import files
from io import BytesIO
from matplotlib import pyplot as plt
from PIL import Image
from PULSE import PULSE
import torchvision
from IPython import display
import numpy as np
from shape_predictor import align_face
from drive import open_url
import dlib
import cv2
from mpl_toolkits.axes_grid1 import ImageGrid
from bicubic import BicubicDownSample

display.clear_output(wait=True)
uploaded_names = files.upload().keys()

f=open_url("https://drive.google.com/uc?id=1huhv8PYpNNKbGCLOaYUjOgR1pY5pmbJx", cache_dir="cache", return_path=True)
predictor = dlib.shape_predictor(f)

toPIL = torchvision.transforms.ToPILImage()
toTensor = torchvision.transforms.ToTensor()
D = 1
while(D==1):
  val = input("Is your picture already pixelated? (Enter Y/N): ")
  if val.lower() == 'y':
    D = BicubicDownSample(factor=int(1))
  elif val.lower() == 'n':
    D = BicubicDownSample(factor=int(32))
  else:
    print()
    print("Invalid Input. Try again with only y and n")
    D = 1


images = []
images_list = []
for x in uploaded_names:
  for y in align_face(x,predictor):
    images_list.append(y)
    y = toPIL(D(toTensor(y).unsqueeze(0).cuda()).cpu().detach().clamp(0,1)[0])
    images.append(y)

model = PULSE(cache_dir="cache", verbose=False)
kwargs={
 'loss_str': '100*L2+0.05*GEOCROSS',
 'seed': None,
 'eps': 1e-3,
 'noise_type': 'trainable',
 'num_trainable_noise_layers': 5,
 'tile_latent': False,
 'bad_noise_layers': '2',
 'opt_name': 'adam',
 'learning_rate': 0.4,
 'steps': 100,
 'lr_schedule': 'linear1cycledrop',
 'save_intermediate': True
}

dims = np.array((len(images),7))
fig = plt.figure(figsize=20*dims)
axs = ImageGrid(fig, 111, nrows_ncols=dims, axes_pad=0.2)

im_downsample = Image.open("resources/downsample.png")
im_PULSE = Image.open("resources/PULSE.png")

display.clear_output(wait=True)
image_list=[]
for ax in axs:
    image_list.append(ax.imshow(Image.new('RGB', (1024,1024), (255, 255, 255))))
    ax.axis('off')

for i,(imHR,imLR) in enumerate(zip(images_list,images)):
    axs[i*dims[1]].imshow(imLR.resize((1024,1024),Image.NEAREST))
    axs[i*dims[1]+1].imshow(im_PULSE)

display.display(plt.gcf())
display.clear_output(wait=True)
start_time = time.time()
times = [0.02]
OG_times = [0.02, 54.5, 110.1, 166.8, 210.6, 269.2]
x_cord = [0,10,20,30,40,50]
for i,PIL_im in enumerate(images):
    ref_im = torchvision.transforms.ToTensor()(PIL_im).unsqueeze(0).cuda()
    for j in range(5):
        running_text = axs[i*dims[1]+j+2].text(50,50,f"Running...",
                                          {'family': 'serif','weight': 'normal','size': 12},
                                          horizontalalignment='left',
                                          verticalalignment='top',
                                          bbox=dict(facecolor='white', alpha=1))
        
        display.display(plt.gcf())
        display.clear_output(wait=True)
        for k,(HR,_) in enumerate(model(ref_im,**kwargs)):
          if(True):
            if(k==0 or (k+1)%10==0):
              PIL_out = toPIL(HR[0].cpu().detach().clamp(0, 1))
              curr_image = image_list[i*dims[1]+j+2].set_data(PIL_out)
              running_text.set_text(f"Running ({k+1}%)")
              display.display(plt.gcf())
              display.clear_output(wait=True)

          if(k+1==kwargs["steps"]):
            PIL_out = toPIL(HR[0].cpu().detach().clamp(0, 1))
            curr_image = image_list[i*dims[1]+j+2].set_data(PIL_out)
            display.display(plt.gcf())
            display.clear_output(wait=True)
          
        running_text.remove()
        display.display(plt.gcf())
        display.clear_output(wait=True)
        times.append(time.time() - start_time)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
plt.plot(x_cord, times, label = "New Model Times")
plt.plot(x_cord, OG_times, label = "Old Model Times")
plt.xlabel("Amount of Pictures")
plt.ylabel("Time (seconds)")
plt.title("Comparison of Models Training Times")
plt.legend()
plt.show()